In [1]:

# Barebones code to plot interactive correlation plot on web

In [20]:
def log(msg):
    with open("voila_debug_log.txt", "a", encoding="utf-8") as f:
        f.write(str(msg) + "\n")

# Optional: clear log at start of each session
with open("voila_debug_log.txt", "w") as f:
    f.write("Starting new Voila session...\n")


In [21]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display
from scipy.stats import pearsonr



In [22]:
# Load the saved CSV file back into a dataframe
merged_data = pd.read_csv("merged_data_MAR_LOI_GS.csv", index_col=0)

In [23]:
# Automatically extract site names and variable types from column names
site_names = sorted(set(col.split('_')[0] for col in merged_data.columns if "_Year" in col))
variables = sorted(set(col.split('_')[1] for col in merged_data.columns if not col.endswith("Year")))


In [25]:
from plotly.offline import plot
from IPython.display import display, HTML
from scipy.stats import pearsonr

# Color-blind-friendly palette (Color Universal Design - CUD)
CUD_COLORS = {
    "LOI": "#E69F00",
    "MAR": "#56B4E9",
    "mean": "#009E73",
    "DBD": "#F0E442",
    "percentclay": "#0072B2",
    "default": "#999999"
}

def log(msg):
    with open("voila_debug_log.txt", "a", encoding="utf-8") as f:
        f.write(str(msg) + "\n")

# Optional: clear the log at the start of the session
with open("voila_debug_log.txt", "w") as f:
    f.write("Starting new Voila session...\n")


# Create the base figure once
fig = go.Figure()

# Add four placeholder traces (background X, background Y, zoomed X, zoomed Y)
fig.add_trace(go.Scatter(name='X (all)', mode='lines+markers', yaxis='y'))
fig.add_trace(go.Scatter(name='Y (all)', mode='lines+markers', yaxis='y2'))
fig.add_trace(go.Scatter(name='X (zoom)', mode='lines+markers', yaxis='y'))
fig.add_trace(go.Scatter(name='Y (zoom)', mode='lines+markers', yaxis='y2'))

# Set up layout here once (same as before)
fig.update_layout(
    xaxis=dict(title='Year'),
    yaxis=dict(
        title=dict(text='Variable 1', font=dict(color='blue')),
        tickfont=dict(color='blue')
    ),
    yaxis2=dict(
        title=dict(text='Variable 2', font=dict(color='red')),
        tickfont=dict(color='red'),
        overlaying='y',
        side='right',
        showgrid=False
    ),
    legend=dict(orientation="h", yanchor="top", y=-0.25, xanchor="center", x=0.5),
    height=500,
)


def update_plot(site, year_range, var1, var2):
    try:
        year_col = f'{site}_Year'
        x_col = f'{site}_{var1}'
        y_col = f'{site}_{var2}'

        df = merged_data[[year_col, x_col, y_col]].dropna()
        df = df.rename(columns={year_col: 'Year', x_col: 'X', y_col: 'Y'})

        df_full = df.copy()
        df_zoom = df[(df['Year'] >= year_range[0]) & (df['Year'] <= year_range[1])]

        if len(df_zoom) >= 2 and df_zoom['X'].nunique() > 1 and df_zoom['Y'].nunique() > 1:
            r, p = pearsonr(df_zoom['X'], df_zoom['Y'])
            corr_label = f"Pearson r = {r:.2f}, p = {p:.2g}"
        else:
            corr_label = "Correlation not valid"

        color1 = CUD_COLORS.get(var1, CUD_COLORS["default"])
        color2 = CUD_COLORS.get(var2, CUD_COLORS["default"])

        # Now just update the traces directly (fig.data)
        fig.data[0].x = df_full['Year']
        fig.data[0].y = df_full['X']
        fig.data[0].name = f'{var1} (all)'
        fig.data[0].line.color = color1
        fig.data[0].line.dash = 'dot'

        fig.data[1].x = df_full['Year']
        fig.data[1].y = df_full['Y']
        fig.data[1].name = f'{var2} (all)'
        fig.data[1].line.color = color2
        fig.data[1].line.dash = 'dot'

        fig.data[2].x = df_zoom['Year']
        fig.data[2].y = df_zoom['X']
        fig.data[2].name = f'{var1} (zoom)'
        fig.data[2].line.color = color1
        fig.data[2].line.width = 4

        fig.data[3].x = df_zoom['Year']
        fig.data[3].y = df_zoom['Y']
        fig.data[3].name = f'{var2} (zoom)'
        fig.data[3].line.color = color2
        fig.data[3].line.width = 4

        # Update titles
        fig.update_layout(
            title=dict(
                text=f"{site}: Variable 1 = {var1}, Variable 2 = {var2} ({year_range[0]}–{year_range[1]})<br>{corr_label}",
                x=0.5,
                xanchor='center'
            ),
            yaxis=dict(title=dict(text=f'Variable 1: {var1}', font=dict(color=color1)), tickfont=dict(color=color1)),
            yaxis2=dict(title=dict(text=f'Variable 2: {var2}', font=dict(color=color2)), tickfont=dict(color=color2)),
        )

        display(HTML(plot(fig, include_plotlyjs=False, output_type='div')))

    except Exception as e:
        log(f"[ERROR] Exception in update_plot: {e}")



In [26]:
site_selector = widgets.Dropdown(
    options=site_names,
    value=site_names[0],
    description='Site:'
)

year_slider = widgets.IntRangeSlider(
    value=[1980, 2000],
    min=1950,
    max=2022,
    step=1,
    description='Year Range:',
    continuous_update=False
)

x_selector = widgets.Dropdown(
    options=variables,
    value='MAR',
    description='X Variable:'
)

y_selector = widgets.Dropdown(
    options=variables,
    value='LOI',
    description='Y Variable:'
)

widgets.interact(
    update_plot,
    site=site_selector,
    year_range=year_slider,
    var1=x_selector,
    var2=y_selector
)


interactive(children=(Dropdown(description='Site:', options=('OC-21-01', 'OC-21-02', 'OC-21-03'), value='OC-21…

<function __main__.update_plot(site, year_range, var1, var2)>